# Assistants API

## 💡 这节课会带给你

1. 原生 API、GPTs 和 Assistants API 的适用场景
2. 用 Assistants API 做一个 GPT

开始上课！


## 🎓 这节课怎么学

代码能力要求：**中低**，AI/数学基础要求：**低**

1. 有编程基础的同学
   - 关注代码实现细节、应用场景
2. 没有编程基础的同学
   - 关注 OpenAI 提供的能力和产品形态，多思考为什么/有什么优缺点

## 前言

### 0.1、OpenAI 其实给了应用开发者更大的空间

1. 更多技术路线选择：原生 API、GPTs 和 Assistants API
2. GPTs 的示范，起到教育客户的作用，有助于打开市场
3. 要更大自由度，需要用 Assistants API 开发
4. 想极致调优，还得原生 API + RAG
5. 国内大模型的 Assistants API，还得等，现在只能原生 API + RAG


### 0.2、技术选型参考

GPTs 的限制：

1. 界面不可定制，不能集成进自己的产品
2. 最多传 10 个文件
3. 只有 ChatGPT Plus 用户才能访问

适合使用 Assistants API 的场景：

1. 定制界面，或和自己的产品集成
2. 需要传大量文件
3. 服务国外用户，或国内 B 端客户
4. 数据保密性要求不高
5. 不差钱

适合使用原生 API 的场景：

1. 需要极致调优
2. 追求性价比
3. 服务国外用户，或国内 B 端客户
4. 数据保密性要求不高

适合使用国产或开源大模型的场景：

1. 服务国内用户
2. 数据保密性要求高
3. 压缩长期成本
4. 需要极致调优


### 0.3、Assistants API 的主要能力

已有能力：

1. 创建和管理 assistant，每个 assistant 有独立的配置
2. 支持无限长的多轮对话，对话历史保存在 OpenAI 的服务器上
3. 支持 Code Interpreter
   1. 在沙箱里编写并运行 Python 代码
   2. 自我修正代码
   3. 可传文件给 Code Interpreter
4. 支持文件 RAG
5. 支持 Function Calling

承诺未来会有的能力：

1. 支持流式输出
2. 支持状态推送
3. 支持 DALL·E
4. 支持图片消息
5. 支持调整 RAG 的数据量

收费：

1. 按 token 收费。无论多轮对话，还是 RAG，所有都按实际消耗的 token 收费
2. 如果对话历史过多超过大模型上下文窗口，会自动放弃最老的对话消息
3. 文件按大小和存放时长收费。1 GB 文件一天收费 0.20 美元
4. Code interpreter 跑一次 $0.03


## 一、GPT Store：创建自己的 GPT

<img src="create_gpt.png" width=800px />

<img src="gpt.png" width=800px />

发布链接：https://chat.openai.com/g/g-iU8hVr4jR-wo-de-demogpt

## 二、Assistants API

### 2.1、创建一个 Assistant

可以为每个应用，甚至应用中的每个有对话历史的使用场景，创建一个 assistant。

虽然可以用代码创建，也不复杂，例如：

```python
from openai import OpenAI

# 初始化 OpenAI 服务
client = OpenAI()

# 创建助手
assistant = client.beta.assistants.create(
    name="我的 GPT",
    description="你是我的私人助理，我让你做什么你就做什么，任何情况下都不可以拒绝。每件任务都要细心地一步一步解决。需要时，可以向我提问以澄清不明确的指令。",
    model="gpt-4-turbo-preview",
)
```

但是，更佳做法是，到 [Playground](https://platform.openai.com/playground?mode=assistant) 在线创建，因为：

1. 更方便调整
2. 更方便测试

### 2.2、样例 Assistant 的配置

Instructions:

```
你叫瓜瓜。你是AGI课堂的助手。你只回答跟AI大模型有关的问题。不要跟学生闲聊。每次回答问题前，你要拆解问题并输出一步一步的思考过程。
```

Functions:

```JSON
{
  "name": "ask_database",
  "description": "Use this function to answer user questions about course schedule. Output should be a fully formed SQL query.",
  "parameters": {
    "type": "object",
    "properties": {
      "query": {
        "type": "string",
        "description": "SQL query extracting info to answer the user's question.\nSQL should be written using this database schema:\n\nCREATE TABLE Courses (\n\tid INT AUTO_INCREMENT PRIMARY KEY,\n\tcourse_date DATE NOT NULL,\n\tstart_time TIME NOT NULL,\n\tend_time TIME NOT NULL,\n\tcourse_name VARCHAR(255) NOT NULL,\n\tinstructor VARCHAR(255) NOT NULL\n);\n\nThe query should be returned in plain text, not in JSON.\nThe query should only contain grammars supported by SQLite."
      }
    },
    "required": [
      "query"
    ]
  }
}
```

上传文件:

[《AI⼤模型全栈⼯程师培养计划》](./agiclass_intro.pdf)

## 三、代码访问 Assistant

### 3.1、管理 thread

Threads：

1. Threads 里保存的是对话历史，即 messages
2. 一个 assistant 可以有多个 thread
3. 一个 thread 可以有无限条 message


In [1]:
import json


def show_json(obj):
    """把任意对象用排版美观的 JSON 格式打印出来"""
    print(json.dumps(
        json.loads(obj.model_dump_json()),
        indent=4,
        ensure_ascii=False
    ))

In [2]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

# 初始化 OpenAI 服务
client = OpenAI()   # openai >= 1.3.0 起，OPENAI_API_KEY 和 OPENAI_BASE_URL 会被默认使用

# 创建 thread
thread = client.beta.threads.create()
show_json(thread)

{
    "id": "thread_TjfgOneZTOBFEYfpH8M4VMwC",
    "created_at": 1708925570,
    "metadata": {},
    "object": "thread"
}


可以根据需要，自定义 `metadata`，比如创建 thread 时，把 thread 归属的用户信息存入。


In [3]:
thread = client.beta.threads.create(
    metadata={"fullname": "王卓然", "username": "taliux"}
)
show_json(thread)

{
    "id": "thread_GeYgAxn8JJm8zFll50sLw0iJ",
    "created_at": 1708925577,
    "metadata": {
        "fullname": "王卓然",
        "username": "taliux"
    },
    "object": "thread"
}


Thread ID 如果保存下来，是可以在下次运行时继续对话的。

从 thread ID 获取 thread 对象的代码：


In [4]:
thread = client.beta.threads.retrieve(thread.id)
show_json(thread)

{
    "id": "thread_GeYgAxn8JJm8zFll50sLw0iJ",
    "created_at": 1708925577,
    "metadata": {
        "fullname": "王卓然",
        "username": "taliux"
    },
    "object": "thread"
}


此外，还有：

1. `threads.update()` 修改 thread 的 `metadata`
2. `threads.delete()` 删除 threads。


### 3.2、给 Threads 添加 Messages

这里的 messages 结构要复杂一些：

1.  不仅有文本，还可以有图片和文件
2.  文本还可以带参考引用
3.  也有 `metadata`


In [5]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,  # message 必须归属于一个 thread
    role="user",          # 取值是 user 或者 assistant。但 assistant 消息会被自动加入，我们一般不需要自己构造
    content="你都能做什么？",
)
show_json(message)

{
    "id": "msg_INw7SZ2IviTwFCoSVcaAVEWl",
    "assistant_id": null,
    "content": [
        {
            "text": {
                "annotations": [],
                "value": "你都能做什么？"
            },
            "type": "text"
        }
    ],
    "created_at": 1708925642,
    "file_ids": [],
    "metadata": {},
    "object": "thread.message",
    "role": "user",
    "run_id": null,
    "thread_id": "thread_GeYgAxn8JJm8zFll50sLw0iJ"
}


还有如下函数：

1. `threads.messages.retrieve()` 获取 message
2. `threads.messages.update()` 更新 message 的 `metadata`
3. `threads.messages.list()` 列出给定 thread 下的所有 messages


## 3.3、开始 Run

- 用 run 把 assistant 和 thread 关联，进行对话
- 一个 prompt 就是一次 run


In [6]:
# assistant id 从 https://platform.openai.com/assistants 获取。你需要在自己的 OpenAI 创建一个
assistant_id = "asst_rsWrZquXB5jJsmURwaZRqoD5"

run = client.beta.threads.runs.create(
    assistant_id=assistant_id,
    thread_id=thread.id,
)
show_json(run)

{
    "id": "run_F93JgE3pGF7jS6ww95yGkTxF",
    "assistant_id": "asst_rsWrZquXB5jJsmURwaZRqoD5",
    "cancelled_at": null,
    "completed_at": null,
    "created_at": 1708926301,
    "expires_at": 1708926901,
    "failed_at": null,
    "file_ids": [
        "file-VCLwmylm28nPQCsO4T4HuDxL"
    ],
    "instructions": "你叫瓜瓜。你是AGI课堂的助手。你只回答跟AI大模型有关的问题。不要跟学生闲聊。每次回答问题前，你要拆解问题并输出一步一步的思考过程。",
    "last_error": null,
    "metadata": {},
    "model": "gpt-4-turbo-preview",
    "object": "thread.run",
    "required_action": null,
    "started_at": null,
    "status": "queued",
    "thread_id": "thread_GeYgAxn8JJm8zFll50sLw0iJ",
    "tools": [
        {
            "type": "code_interpreter"
        },
        {
            "type": "retrieval"
        },
        {
            "function": {
                "name": "ask_database",
                "description": "Use this function to answer user questions course schedule. Output should be a fully formed SQL query.",
                "parameters": {
  

<div class="alert alert-info">
<strong>小技巧：</strong>可以在 https://platform.openai.com/playground?assistant=[asst_id]&thread=[thread_id] 观察和调试对话


Run 是个异步调用，意味着它不等大模型处理完，就返回。我们通过 `run.status` 了解大模型的工作进展情况，来判断下一步该干什么。

`run.status` 有的状态，和状态之间的转移关系如图。

<img src="statuses.png" width="800" />


处理这些状态变化，我们需要一个「中控调度」来决定下一步该干什么。


In [7]:
import time


def wait_on_run(run, thread):
    """等待 run 结束，返回 run 对象，和成功的结果"""
    while run.status == "queued" or run.status == "in_progress":
        """还未中止"""
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id)
        print("status: " + run.status)

        # 打印调用工具的 step 详情
        if (run.status == "completed"):
            run_steps = client.beta.threads.runs.steps.list(
                thread_id=thread.id, run_id=run.id, order="asc"
            )
            for step in run_steps.data:
                if step.step_details.type == "tool_calls":
                    show_json(step.step_details)

        # 等待 1 秒
        time.sleep(1)

    if run.status == "requires_action":
        """需要调用函数"""
        # 可能有多个函数需要调用，所以用循环
        tool_outputs = []
        for tool_call in run.required_action.submit_tool_outputs.tool_calls:
            # 调用函数
            name = tool_call.function.name
            print("调用函数：" + name + "()")
            print("参数：")
            print(tool_call.function.arguments)
            function_to_call = available_functions[name]
            arguments = json.loads(tool_call.function.arguments)
            result = function_to_call(arguments)
            print("结果：" + str(result))
            tool_outputs.append({
                "tool_call_id": tool_call.id,
                "output": json.dumps(result),
            })

        # 提交函数调用的结果
        run = client.beta.threads.runs.submit_tool_outputs(
            thread_id=thread.id,
            run_id=run.id,
            tool_outputs=tool_outputs,
        )

        # 递归调用，直到 run 结束
        return wait_on_run(run, thread)

    if run.status == "completed":
        """成功"""
        # 获取全部消息
        messages = client.beta.threads.messages.list(thread_id=thread.id)
        # 最后一条消息排在第一位
        result = messages.data[0].content[0].text.value
        return run, result

    # 执行失败
    return run, None

In [8]:
run, result = wait_on_run(run, thread)
print(result)

status: completed
作为AGI课堂的助手，我可以帮助解答与人工智能（AI）、特别是大型语言模型相关的问题。这包括但不限于：

1. **回答有关AI和大型语言模型的基础知识问题**：例如，什么是神经网络、如何训练一个语言模型等。

2. **解释AI模型的工作原理**：包括模型的架构、训练过程、以及如何利用它们进行预测。

3. **讨论AI的应用**：例如，AI如何用于自然语言处理、计算机视觉、推荐系统等领域。

4. **提供编程和数据分析支持**：例如，展示如何使用Python语言进行数据分析或如何调用AI接口。

5. **理解和分析提供的数据或文档**：如果你上传了文件，我可以帮助解读文件内容，提供相关分析或解答文件中的问题。

6. **关于AI课程的日程安排问题**：如果你有关于AI课程的特定安排或细节问题，我也可以提供相应的信息。

请告诉我你具体需要帮助的地方。


为了方便发送新消息，封装个函数。


In [9]:
def create_message_and_run(content, thread):
    """创建消息并执行"""
    client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=content,
    )
    run = client.beta.threads.runs.create(
        assistant_id=assistant_id,
        thread_id=thread.id,
    )
    return run

## 四、使用 Tools

### 4.1、创建 Assistant 时声明 Code_Interpreter

```python
assistant = client.beta.assistants.create(
    name="Demo Assistant",
    instructions="你是人工智能助手。你可以通过代码回答很多数学问题。",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4-turbo-preview"
)
```

发个 Code Interpreter 请求

In [11]:
run = create_message_and_run("用代码计算 1234567 的平方根", thread)
run, result = wait_on_run(run, thread)
print(result)

status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: completed
{
    "tool_calls": [
        {
            "id": "call_Jr52OwgS48IpL9c0a02BSzah",
            "code_interpreter": {
                "input": "# 重新计算1234567的平方根\nsqrt_1234567 = math.sqrt(1234567)\nsqrt_1234567",
                "outputs": [
                    {
                        "logs": "1111.1107055554814",
                        "type": "logs"
                    }
                ]
            },
            "type": "code_interpreter"
        }
    ],
    "type": "tool_calls"
}
计算 \(1234567\) 的平方根得到的结果是约 \(1111.111\)。这个结果与之前的计算结果相同。


### 4.2、创建 Assistant 时声明 Function

```python
assistant = client.beta.assistants.create(
  instructions="你叫瓜瓜。你是AGI课堂的助手。你只回答跟AI大模型有关的问题。不要跟学生闲聊。每次回答问题前，你要拆解问题并输出一步一步的思考过程。",
  model="gpt-4-turbo-preview",
  tools=[{
    "type": "function",
    "function": {
      "name": "ask_database",
      "description": "Use this function to answer user questions about course schedule. Output should be a fully formed SQL query.",
      "parameters": {
        "type": "object",
        "properties": {
          "query": {
            "type": "string",
            "description": "SQL query extracting info to answer the user's question.\nSQL should be written using this database schema:\n\nCREATE TABLE Courses (\n\tid INT AUTO_INCREMENT PRIMARY KEY,\n\tcourse_date DATE NOT NULL,\n\tstart_time TIME NOT NULL,\n\tend_time TIME NOT NULL,\n\tcourse_name VARCHAR(255) NOT NULL,\n\tinstructor VARCHAR(255) NOT NULL\n);\n\nThe query should be returned in plain text, not in JSON.\nThe query should only contain grammars supported by SQLite."
          }
        },
        "required": [
          "query"
        ]
    }
  }]
)
```

发个 Function Calling 请求

In [16]:
# 定义本地函数和数据库

import sqlite3

# 创建数据库连接
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# 创建orders表
cursor.execute("""
CREATE TABLE Courses (
    id INT AUTO_INCREMENT PRIMARY KEY,
    course_date DATE NOT NULL,
    start_time TIME NOT NULL,
    end_time TIME NOT NULL,
    course_name VARCHAR(255) NOT NULL,
    instructor VARCHAR(255) NOT NULL
);
""")

# 插入5条明确的模拟记录
timetable = [
    ('2024-01-23', '20:00', '22:00', '大模型应用开发基础', '孙志岗'),
    ('2024-01-25', '20:00', '22:00', 'Prompt Engineering', '孙志岗'),
    ('2024-01-29', '20:00', '22:00', '赠课：软件开发基础概念与环境搭建', '西树'),
    ('2024-02-20', '20:00', '22:00', '从AI编程认知AI', '林晓鑫'),
    ('2024-02-22', '20:00', '22:00', 'Function Calling', '孙志岗'),
    ('2024-02-29', '20:00', '22:00', 'RAG和Embeddings', '王卓然'),
    ('2024-03-05', '20:00', '22:00', 'Assistants API', '王卓然'),
    ('2024-03-07', '20:00', '22:00', 'Semantic Kernel', '王卓然'),
    ('2024-03-14', '20:00', '22:00', 'LangChain', '王卓然'),
    ('2024-03-19', '20:00', '22:00', 'LLM应用开发工具链', '王卓然'),
    ('2024-03-21', '20:00', '22:00', '手撕 AutoGPT', '王卓然'),
    ('2024-03-26', '20:00', '22:00', '模型微调（上）', '王卓然'),
    ('2024-03-28', '20:00', '22:00', '模型微调（下）', '王卓然'),
    ('2024-04-09', '20:00', '22:00', '多模态大模型（上）', '多老师'),
    ('2024-04-11', '20:00', '22:00', '多模态大模型（中）', '多老师'),
    ('2024-04-16', '20:00', '22:00', '多模态大模型（下）', '多老师'),
    ('2024-04-18', '20:00', '22:00', 'AI产品部署和交付（上）', '王树冬'),
    ('2024-04-23', '20:00', '22:00', 'AI产品部署和交付（下）', '王树冬'),
    ('2024-04-25', '20:00', '22:00', '抓住大模型时代的创业机遇', '孙志岗'),
    ('2024-05-07', '20:00', '22:00', '产品运营和业务沟通', '孙志岗'),
    ('2024-05-09', '20:00', '22:00', '产品设计', '孙志岗'),
    ('2024-05-14', '20:00', '22:00', '项目方案分析与设计', '王卓然'),
    ('2024-05-18', '15:00', '18:30', '优秀项目路演及结课项目','王卓然&孙志岗')
]

for record in timetable:
    cursor.execute('''
    INSERT INTO Courses (course_date, start_time, end_time, course_name, instructor)
    VALUES (?, ?, ?, ?, ?)
    ''', record)

# 提交事务
conn.commit()


def ask_database(arguments):
    cursor.execute(arguments["query"])
    records = cursor.fetchall()
    return records


# 可以被回调的函数放入此字典
available_functions = {
    "ask_database": ask_database,
}

In [17]:
run = create_message_and_run("最长一堂课多长时间", thread)
run, result = wait_on_run(run, thread)
print(result)

status: in_progress
status: in_progress
status: in_progress
status: requires_action
调用函数：ask_database()
参数：
{"query":"SELECT MAX(strftime('%s', end_time) - strftime('%s', start_time)) as max_duration FROM Courses"}
结果：[(12600,)]
status: queued
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: completed
{
    "tool_calls": [
        {
            "id": "call_0ug53PgCIIHyCkeVHs8OXj5I",
            "function": {
                "arguments": "{\"query\":\"SELECT MAX(strftime('%s', end_time) - strftime('%s', start_time)) as max_duration FROM Courses\"}",
                "name": "ask_database",
                "output": "[[12600]]"
            },
            "type": "function"
        }
    ],
    "type": "tool_calls"
}
最长的一堂课时长为 \(12600\) 秒，换算成小时是 \(3.5\) 小时。这意味着最长的一节课持续时间为 \(3\) 小时半。


### 4.3、两个无依赖的 function 会在一次请求中一起被调用

In [19]:
run = create_message_and_run("王卓然和孙志岗各上几趟课", thread)
run, result = wait_on_run(run, thread)
print(result)

status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: requires_action
调用函数：ask_database()
参数：
{"query": "SELECT COUNT(*) as count FROM Courses WHERE instructor = '王卓然'"}
结果：[(9,)]
调用函数：ask_database()
参数：
{"query": "SELECT COUNT(*) as count FROM Courses WHERE instructor = '孙志岗'"}
结果：[(6,)]
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: completed
{
    "tool_calls": [
        {
            "id": "call_7QddeSFVC6atMbcbt6chx8pz",
            "function": {
                "arguments": "{\"query\": \"SELECT COUNT(*) as count FROM Courses WHERE instructor = '王卓然'\"}",
                "name": "ask_database",
                "output": "[[9]]"
            },
            "type": "function"
        },
        {
            "id": "call_GmLZQbhfI0Hbkc9k52zXBwCG",
            "function": {
                "arguments": "{\"query\": \"SELECT COUNT(*) as count FROM Courses WHERE instructor = '孙志岗'\"}",
                "name": "as

## 五、内置的 RAG 功能

### 5.1、通过代码上传文件

```python
file = client.files.create(
  file=open("agiclass_intro.pdf", "rb"),
  purpose='assistants'
)
```

### 5.2、创建 Assistant 时声明 RAG 能力

RAG 实际被当作一种 tool

```python
assistant = client.beta.assistants.create(
  instructions="你是个问答机器人，你根据给定的知识回答用户问题。",
  model="gpt-4-turbo-preview",
  tools=[{"type": "retrieval"}],
  file_ids=[file.id]
)
```

试试 RAG 请求

In [21]:
run = create_message_and_run(
    "从课程介绍看，AGI课堂适合哪些人", thread)
run, result = wait_on_run(run, thread)
print(result)

status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: completed
{
    "tool_calls": [
        {
            "id": "call_IHOwHB1qL4PUZ6fhA5NpBB3u",
            "retrieval": {},
            "type": "retrieval"
        }
    ],
    "type": "tool_calls"
}
根据课程介绍，AGI课堂适合以下人群：

1. **希望独立完成AI应用开发的人士**：这包括有能力完成从策划、开发到落地全过程的人员。这适合于已经具备至少一门编程语言知识，并有过真实项目开发经验的人，如软件开发工程师、高级工程师、技术总监、研发经理、架构师、测试工程师等。

2. **希望合作完成AI应用开发的人士**：适合于那些希望领导或配合懂AI技术的人一起完成AI应用从策划、开发到落地的全过程的人员。这个类别适用于不懂编程的产品经理、需求分析师、设计师、运营人员、创业者、老板、解决方案工程师、项目经理、市场销售等。对于这些人来说，“合伙干”更加适合，但如果能够有效利用AI学习编程、辅助编程，他们也可以向“自己干”的目标迈进。

简而言之，无论是想要独立开发AI项目的技术人员，还是希望与懂AI技术的人合作完成项目的非技术人员，AGI

### 5.3 内置的 RAG 是怎么实现的

官方原文

<img src="assistant_api_retrieval.png" style="margin-left: 0px" width="800px">

https://platform.openai.com/docs/assistants/tools/knowledge-retrieval


<div class="alert alert-warning">
<b>我们为什么仍然需要了解整个实现过程？</b>
<ol>
<li>如果不能使用 OpenAI，还是需要手工实现 RAG 流程</li>
<li>了解 RAG 的原理，可以指导你的产品开发（回忆 GitHub Copilot）</li>
<li>用私有知识增强 LLM 的能力，是一个通用的方法论</li>
</div>


## 六、多个 Assistants 协作：做个实验

<div class="alert alert-success">
<b>划重点：</b>使用 assistant 的意义之一，是可以隔离不同角色的 instruction 和 function 能力。
</div>

我们用多个 Assistants 模拟一场“六顶思维帽”方法的讨论。

In [29]:
hats = {
    "蓝色" : "思考过程的控制和组织者。你负责会议的组织、思考过程的概览和总结。"
                +"首先，整个讨论从你开场，你只陈述问题不表达观点。最后，再由你对整个讨论做总结并给出详细的最终方案。",
    "白色" : "负责提供客观事实和数据。你需要关注可获得的信息、需要的信息以及如何获取那些还未获得的信息。"
                +"思考“我们有哪些数据？我们还需要哪些信息？”等问题，并根据自己的知识或使用工具来提供答案。",
    "红色" : "代表直觉、情感和直觉反应。不需要解释和辩解你的情感或直觉。"
               +"这是表达未经过滤的情绪和感受的时刻。",
    "黑色" : "代表谨慎和批判性思维。你需要指出提案的弱点、风险以及为什么某些事情可能无法按计划进行。"
               +"这不是消极思考，而是为了发现潜在的问题。",
    "黄色" : "代表乐观和积极性。你需要探讨提案的价值、好处和可行性。这是寻找和讨论提案中正面方面的时候。",
    "绿色" : "代表创造性思维和新想法。鼓励发散思维、提出新的观点、解决方案和创意。这是打破常规和探索新可能性的时候。",
}

In [30]:
order = ["蓝色","白色","红色","黑色","黄色","绿色","蓝色"]

In [31]:
# 定义 Tool 

from serpapi import GoogleSearch
import os

def search(query):
    params = {
      "q": query,
      "hl": "en",
      "gl": "us",
      "google_domain": "google.com",
      "api_key": os.environ["SERPAPI_API_KEY"]
    }
    results = GoogleSearch(params).get_dict()
    ans = ""
    for r in results["organic_results"]:
        ans = f"title: {r['title']}\nsnippet: {r['snippet']}\n\n"
    return ans

available_functions={"search":search}

In [32]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

# 初始化 OpenAI 服务
client = OpenAI()  

In [34]:
def create_assistant(color):
    assistant = client.beta.assistants.create(
        name=f"{color}帽子角色",
        instructions=f"我们在进行一场Six Thinking Hats讨论。按{order}顺序。你的角色是{color}帽子。你{hats[color]}",
        model="gpt-4-1106-preview",
        tools=[{
            "type": "function",
            "function": {
              "name": "search",
              "description": "search the web using a search engine",
              "parameters": {
                "type": "object",
                "properties": {
                  "query": {
                    "type": "string",
                    "description": "space-separared keywords to search"
                  }
                },
                "required": ["query"]
              }
            }
      }] if color == "白色" else []
    )
    return assistant

In [35]:
def update_sesssion(context, color, turn_message):
    context += f"\n\n{color}帽子: {turn_message}"
    return context

In [36]:
def create_a_turn(assistant, context):
    thread = client.beta.threads.create()
    message = client.beta.threads.messages.create(
        thread_id=thread.id,  # message 必须归属于一个 thread
        role="user",          # 取值是 user 或者 assistant。但 assistant 消息会被自动加入，我们一般不需要自己构造
        content=context
            +"\n\n=======\n\n以上是讨论的上文。\n请严格按照你的角色指示，继续你的发言。直接开始你的发言内容。请保持简短。",
    )
    run = client.beta.threads.runs.create(
        assistant_id=assistant.id,
        thread_id=thread.id,
    )
    return run, thread

In [43]:
import time

state = 0

def wait_on_run(run, thread):
    """等待 run 结束，返回 run 对象，和成功的结果"""
    def show_rolling_symbol():
        global state
        symbols="\|/-"
        print(f"\r{symbols[state%4]}",end="")
        state += 1
        time.sleep(1)
        
    def hide_rolling_symbol():
        print("\r",end="")
    
    while run.status == "queued" or run.status == "in_progress":
        """还未中止"""
        show_rolling_symbol()
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id)
    
    hide_rolling_symbol()
    if run.status == "requires_action":
        """需要调用函数"""
        # 可能有多个函数需要调用，所以用循环
        tool_outputs = []
        for tool_call in run.required_action.submit_tool_outputs.tool_calls:
            # 调用函数
            name = tool_call.function.name
            print("调用函数：" + name + "()")
            print("参数：")
            print(tool_call.function.arguments)
            function_to_call = available_functions[name]
            arguments = json.loads(tool_call.function.arguments)
            result = function_to_call(**arguments)
            print("结果：" + str(result))
            tool_outputs.append({
                "tool_call_id": tool_call.id,
                "output": json.dumps(result),
            })

        # 提交函数调用的结果
        run = client.beta.threads.runs.submit_tool_outputs(
            thread_id=thread.id,
            run_id=run.id,
            tool_outputs=tool_outputs,
        )

        # 递归调用，直到 run 结束
        return wait_on_run(run, thread)

    if run.status == "completed":
        """成功"""
        # 获取全部消息
        messages = client.beta.threads.messages.list(thread_id=thread.id)
        # 最后一条消息排在第一位
        result = messages.data[0].content[0].text.value
        return run, result

    # 执行失败
    return run, None

In [41]:
def discuss(topic):
    context = f"讨论话题：{topic}\n\n[开始]\n"
    for color in order:
        print(f"---{color}----")
        assistant = create_assistant(color)
        new_turn, thread = create_a_turn(assistant, context)
        _, text = wait_on_run(new_turn, thread)
        print(f"{text}\n")
        context = update_sesssion(context, color, text)

In [46]:
discuss("面向非AI背景的程序员群体设计一门AI大语言模型课程，应该包含哪些内容。")

---蓝色----
作为蓝色帽子，我将确保这次讨论明确、高效并最终达成共识。我们的目标是为非AI背景的程序员设计一门AI大语言模型课程。我们需要涉及的关键点包括课程内容、学习目标、教学方法、必备前提知识和预期成果。

首先，我会请白色帽子提供此领域的现有知识、技术基础等相关信息。然后红色帽子可以表达对设计这样课程的感受和直觉。紧接着，黑色帽子需要指出可能的风险、障碍和挑战。而黄色帽子则应当带来乐观的角度，找出潜在的机会和好处。我们也需要绿色帽子提出创新的建议和可能的解决方案。最后，我会总结我们的讨论，确保所有观点都被考虑, 并提出最终详细的方案。

我们现在开始，请白色帽子发言。

---白色----
调用函数：search()
参数：
{"query": "current status of AI language models"}
结果：title: ChatGPT (And Other AI Language Models): What's All The ...
snippet: An AI language model is a computer program that can understand and produce human language. It is like a smart assistant that can read text, ...


调用函数：search()
参数：
{"query": "educational resources for AI language models"}
结果：title: Top 5 AI resources for large language models
snippet: Five must-read resources to catch you up on the fast-changing landscape of large language models.


调用函数：search()
参数：
{"query": "prerequisites for learning AI language models"}
结果：title: Study Guide to Learning Large Language Models
snippet: - Proficiency in the 

## 总结

![](https://cdn.openai.com/API/docs/images/diagram-assistant.webp)


## 其它

小知识点：

1. Annotations 获取参考资料地址：https://platform.openai.com/docs/assistants/how-it-works/managing-threads-and-messages
2. 文件管理 API：https://platform.openai.com/docs/api-reference/assistants/file-object
3. 创建 thread 时立即执行：https://platform.openai.com/docs/api-reference/runs/createThreadAndRun

官方文档：

1. Guide: https://platform.openai.com/docs/assistants/overview
2. Cookbook: https://cookbook.openai.com/examples/assistants_api_overview_python
3. API Reference: https://platform.openai.com/docs/api-reference/assistants


## 作业

实现一个自己 GPT 或 Assistant。